In [1]:
!pip install twarc --user
!pip install twarc_csv --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from twarc import Twarc2, expansions
import datetime
import json
from twarc_csv import CSVConverter
import time
import pandas as pd
import pytz
import os
import glob

/home/jeana/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Count Tweets Function

In [3]:
def count_tweet_results(query, start_time, end_time, bearer_token, hashtag):
    client = Twarc2(bearer_token=bearer_token)
    count_results = client.counts_all(query=query, start_time=start_time, end_time=end_time)
    json_file = hashtag + '_' + start_time.strftime('%Y%m%d') + ".jsonl"
    
    with open(json_file, "w+", encoding="utf-8") as f:
        for page in count_results:
            f.write('%s\n' % json.dumps(page))

# Main Count Query

In [4]:
# query = "-is:retweet lang:tl (halalan OR election OR eleksyon OR comelec OR PPCRV)"
# query = '-is:retweet lang:tl (halalan OR election OR eleksyon OR comelec OR PPCRV) ("takot ako" OR "takot ko" OR "sindak ako" OR "ngamba ako" OR "ngamba ko" OR "katakot" OR "galit ako" OR "galit ko" OR "gigil ako" OR "qiqil ako" OR "qiqil ko" OR "kainis" OR "inis ko" OR "inis ako" OR "kagalit" OR "kagigil" OR "tuwa ako" OR "tuwa ko" OR "saya ako" OR "saya ko" OR "katuwa" OR "nasayahan ako" OR "nasiyahan ako" OR "lungkot ako" OR "lungkot ko" OR "iyak ako" OR "iyak ko" OR "kaiyak" OR "katuwa" OR "diri ako" OR "kadiri" OR "ew" OR "yuck" OR "disgust" OR "gulat ako" OR "gulat ko" OR "kagulat" OR "mangha ako" OR "mangha ko" OR "bigla ako" OR "bigla ko" OR "surprise ako" OR "sorpresa ako")'
query = '-is:retweet lang:tl (halalan OR election OR eleksyon OR comelec OR PPCRV) (takot OR sindak OR ngamba OR katakot OR galit OR gigil OR qiqil OR inis OR tuwa OR saya OR tawa OR halakhak OR tumawa OR sumaya OR humalakhak OR lungkot OR iyak OR diri OR yuck OR disgust OR gulat OR mangha OR bigla OR suprise OR sorpresa)'

# Specify the start time in UTC for the time period you want Tweets from
start_time = datetime.datetime(2021, 9, 1, 8, 0, 0, 0, datetime.timezone.utc)

# Specify the end time in UTC for the time period you want Tweets from
end_time = start_time + datetime.timedelta(days=1)

bearer_token = "provide bearer token"

hashtag = 'elections_emotions'

count_tweet_results(query, start_time, end_time, bearer_token, hashtag)

In [5]:
query

'-is:retweet lang:tl (halalan OR election OR eleksyon OR comelec OR PPCRV) (takot OR sindak OR ngamba OR katakot OR galit OR gigil OR qiqil OR inis OR tuwa OR saya OR tawa OR halakhak OR tumawa OR sumaya OR humalakhak OR lungkot OR iyak OR diri OR yuck OR disgust OR gulat OR mangha OR bigla OR suprise OR sorpresa)'

In [6]:
# #Query multiple times through for loop

# extract_number_days = 267
# file_list = []

# for i in range(extract_number_days):
#     #one run of the tweets extraction
#     count_tweet_results(query, start_time, end_time, bearer_token, hashtag)
    
#     json_file = hashtag + '_' + start_time.strftime('%Y%m%d') + ".jsonl"
#     file_list.append(json_file)
    
#     start_time = end_time
#     end_time = start_time + datetime.timedelta(days=1)

In [7]:
# #Complile all jsonl count results into one csv file
# dfs = []
# for file in file_list:
#     with open(file) as f:
#         for line in f:
#             json_data = pd.json_normalize(json.loads(line))
#             dfs.append(json_data)
# df = pd.concat(dfs, sort=False)

# df.to_csv('total_tweet_count.csv')

# Tweet Extraction Functions

In [8]:
from twarc import Twarc2, expansions
import datetime
import json
from twarc_csv import CSVConverter
import time
import pandas as pd
import pytz
import os
import glob

In [9]:
def extract_tweets_to_json_limited(search_results, json_file, Num_pages_to_extract, num_tweets_to_extract, stop_when):
    page_counter = 0
    tweet_counter = 0
    if stop_when == 'pages':
        
        # Print the header of the result table
        print("-"*20)
        print(f"{'Page':^7} | {'Elapsed':^9}")
        print("-"*20)
        
        for page in search_results:
            t0 = time.time()
            # The Twitter API v2 returns the Tweet information and the user, media etc.  separately
            # so we use expansions.flatten to get all the information in a single JSON
            result = expansions.flatten(page)

            with open(json_file, "a+", encoding="utf-8") as f:
                for tweet in result:
                    f.write('%s\n' % json.dumps(tweet))

            page_counter +=1
            time_elapsed = time.time() - t0
            
            print(f"{page_counter:^7} | {time_elapsed:^9.2f}")
            print("-"*20)
            
            # t0 = time.time()
            
            if page_counter >= Num_pages_to_extract:
                print("Finished writing {} page/s of results...".format(Num_pages_to_extract))
                break
    else:
        for page in search_results:
            # The Twitter API v2 returns the Tweet information and the user, media etc.  separately
            # so we use expansions.flatten to get all the information in a single JSON
            result = expansions.flatten(page)

            with open(json_file, "a+", encoding="utf-8") as f:
                for tweet in result:
                    f.write('%s\n' % json.dumps(tweet))
                    tweet_counter += 1
                    if tweet_counter >= num_tweets_to_extract:
                        print('\n')
                        print("Finished writing {} tweet/s of results...".format(num_tweets_to_extract))
                        break
                else:
                    continue
                break          

def extract_tweets_to_json_all(search_results, json_file, Num_pages_to_extract, num_tweets_to_extract, stop_when):
    page_counter = 0
    tweet_counter = 0
    if stop_when == 'pages':
        
        # Print the header of the result table
        print("-"*20)
        print(f"{'Page':^7} | {'Elapsed':^9}")
        print("-"*20)
        
        for page in search_results:
            t0 = time.time()
            # The Twitter API v2 returns the Tweet information and the user, media etc.  separately
            # so we use expansions.flatten to get all the information in a single JSON
            result = expansions.flatten(page)

            with open(json_file, "a+", encoding="utf-8") as f:
                for tweet in result:
                    f.write('%s\n' % json.dumps(tweet))

            page_counter +=1
            time_elapsed = time.time() - t0
            
            print(f"{page_counter:^7} | {time_elapsed:^9.2f}")
            print("-"*20)
            
            # t0 = time.time()
            
        
        print("Finished writing {} page/s of results...".format(page_counter))
                   
                
def convert_json_to_csv(json_file, csv_file):
    print("Start converting json to csv...")
    with open(json_file, "r", encoding="utf-8") as infile:
        with open(csv_file, "w", encoding="utf-8") as outfile:
            converter = CSVConverter(infile, outfile)
            converter.process()

    print("Finished.")
    
def convert_json_to_csv_v2(json_file, csv_file):
    print("Start converting json to csv...")
    with open(json_file, "r", encoding="utf-8") as infile:
        df = pd.read_json(infile, lines=True)
        df.to_csv(csv_file) 
    print("Finished.") 
    
def tweet_query(client, query, start_time, end_time, max_results, json_file, csv_file, Num_pages_to_extract
                , num_tweets_to_extract, stop_when, extract_type = 'limited'):
    
    # The search_all method call the full-archive search endpoint to get Tweets based on the query, start and end times
    search_results = client.search_all(query=query, start_time=start_time, end_time=end_time, max_results=max_results)
    
    print('searching tweets...')
    
    if extract_type == 'limited':
        extract_tweets_to_json_limited(search_results, json_file, Num_pages_to_extract, num_tweets_to_extract, stop_when)
    else:
        extract_tweets_to_json_all(search_results, json_file, Num_pages_to_extract, num_tweets_to_extract, stop_when)
    
#     convert_json_to_csv(json_file, csv_file)
    convert_json_to_csv_v2(json_file, csv_file)
        

# Main Extraction Query

In [10]:
# query = '-is:retweet lang:tl (halalan OR election OR eleksyon OR comelec OR PPCRV) ("takot ako" OR "takot ko" OR "sindak ako" OR "ngamba ako" OR "ngamba ko" OR "katakot" OR "galit ako" OR "galit ko" OR "gigil ako" OR "qiqil ako" OR "qiqil ko" OR "kainis" OR "inis ko" OR "inis ako" OR "kagalit" OR "kagigil" OR "tuwa ako" OR "tuwa ko" OR "saya ako" OR "saya ko" OR "katuwa" OR "nasayahan ako" OR "nasiyahan ako" OR "lungkot ako" OR "lungkot ko" OR "iyak ako" OR "iyak ko" OR "kaiyak" OR "katuwa" OR "diri ako" OR "kadiri" OR "ew" OR "yuck" OR "disgust" OR "gulat ako" OR "gulat ko" OR "kagulat" OR "mangha ako" OR "mangha ko" OR "bigla ako" OR "bigla ko" OR "surprise ako" OR "sorpresa ako")'
# query = '-is:retweet lang:tl (halalan OR election OR eleksyon OR comelec OR PPCRV) (takot OR sindak OR ngamba OR katakot OR galit OR gigil OR qiqil OR inis OR tuwa OR saya OR tawa OR halakhak OR tumawa OR sumaya OR humalakhak OR lungkot OR iyak OR diri OR yuck OR disgust OR gulat OR mangha OR bigla OR suprise OR sorpresa)'
# query = '-is:retweet lang:tl (halalan OR election OR eleksyon OR comelec OR PPCRV) (leni OR bbm OR lacson OR isko OR pacquiao)'
# query = '-is:retweet lang:tl (halalan OR election OR eleksyon OR comelec OR PPCRV)  @GrowlHarhar @Charlbumbum @MDSOnwardPH22 @dragun214 @dennisquijada4 @fugitivemind_ @Akoganda12 @BuangSonny @Disisyiti @GraceEverlast @ces_riki @lenirobredo @FedicTheGreat @MarionA56n @Du30Titan @marcosdaily_ph @srsasot_ @iamkarendavila @inquirerdotnet @PhilippineStar @tribunephl @n4n4daim3 @DapatSiLeni @AdeWinter8 @BBMSigawNgBayan @COMELEC @jabjimenez @rowena_guanzon @philstar '
query = '-is:retweet lang:tl (halalan OR election OR eleksyon OR comelec OR PPCRV)'

In [11]:
# Bearer Token

# Specify the start time in UTC for the time period you want Tweets from
start_time = datetime.datetime(2021, 9, 1, 8, 0, 0, 0, datetime.timezone.utc)

# Specify the end time in UTC for the time period you want Tweets from
end_time = datetime.datetime(2022, 5, 26, 8, 0, 0, 0, datetime.timezone.utc)
# end_time = start_time + datetime.timedelta(days=30)

#Specify hashtag or specify keyword
hashtag = 'election_all_data'

#max results per page
max_results = 100

#Specify how many pages to extract tweets 
Num_pages_to_extract = 1

#OR specify how many tweets to extract
num_tweets_to_extract = 100

#Choose to stop when either number of pages or number of tweets are reached
stop_when = 'pages' #'tweets' or 'pages'; pages option is more efficient (6 times faster)

#Name of json file wherein the tweets are stored
json_file = hashtag + '_' + start_time.strftime('%Y%m%d') + ".jsonl" #UkraineRussiaWar_20220221.jsonl

#Name of csv file wherein tweets will be stored after converting from json format
csv_file = hashtag + '_' + start_time.strftime('%Y%m%d') + ".csv" #UkraineRussiaWar_20220221.csv

#initialize client search
client = Twarc2(bearer_token= bearer_token)

In [ ]:
tweet_query(client, query, start_time, end_time, max_results, json_file, csv_file, Num_pages_to_extract
                , num_tweets_to_extract, stop_when, extract_type = 'all')

searching tweets...
--------------------
 Page   |  Elapsed 
--------------------
   1    |   0.03   
--------------------
   2    |   0.03   
--------------------
   3    |   0.03   
--------------------
   4    |   0.03   
--------------------
   5    |   0.03   
--------------------
   6    |   0.03   
--------------------
   7    |   0.03   
--------------------
   8    |   0.03   
--------------------
   9    |   0.03   
--------------------
  10    |   0.03   
--------------------
  11    |   0.03   
--------------------
  12    |   0.03   
--------------------
  13    |   0.03   
--------------------
  14    |   0.03   
--------------------
  15    |   0.03   
--------------------
  16    |   0.04   
--------------------
  17    |   0.03   
--------------------
  18    |   0.03   
--------------------
  19    |   0.02   
--------------------
  20    |   0.03   
--------------------
  21    |   0.03   
--------------------
  22    |   0.03   
--------------------
  23    |   0.03

rate limit exceeded: sleeping 47.759376525878906 secs


  601   |   0.03   
--------------------
  602   |   0.03   
--------------------
  603   |   0.03   
--------------------
  604   |   0.03   
--------------------
  605   |   0.03   
--------------------
  606   |   0.03   
--------------------
  607   |   0.02   
--------------------
  608   |   0.03   
--------------------
  609   |   0.03   
--------------------
  610   |   0.03   
--------------------
  611   |   0.03   
--------------------
  612   |   0.02   
--------------------
  613   |   0.02   
--------------------
  614   |   0.02   
--------------------
  615   |   0.03   
--------------------
  616   |   0.03   
--------------------
  617   |   0.03   
--------------------
  618   |   0.03   
--------------------
  619   |   0.03   
--------------------
  620   |   0.02   
--------------------
  621   |   0.03   
--------------------
  622   |   0.02   
--------------------
  623   |   0.02   
--------------------
  624   |   0.03   
--------------------
  625   |   0.03

rate limit exceeded: sleeping 43.30367183685303 secs


  901   |   0.02   
--------------------
  902   |   0.04   
--------------------
  903   |   0.03   
--------------------
  904   |   0.03   
--------------------
  905   |   0.03   
--------------------
  906   |   0.03   
--------------------
  907   |   0.02   
--------------------
  908   |   0.03   
--------------------
  909   |   0.03   
--------------------
  910   |   0.03   
--------------------
  911   |   0.02   
--------------------
  912   |   0.03   
--------------------
  913   |   0.02   
--------------------
  914   |   0.02   
--------------------
  915   |   0.02   
--------------------
  916   |   0.02   
--------------------
  917   |   0.02   
--------------------
  918   |   0.02   
--------------------
  919   |   0.03   
--------------------
  920   |   0.02   
--------------------
  921   |   0.03   
--------------------
  922   |   0.03   
--------------------
  923   |   0.03   
--------------------
  924   |   0.02   
--------------------
  925   |   0.02

...

 1501   |   0.02   
--------------------
 1502   |   0.02   
--------------------
 1503   |   0.02   
--------------------
 1504   |   0.02   
--------------------
 1505   |   0.02   
--------------------
 1506   |   0.02   
--------------------
 1507   |   0.02   
--------------------
 1508   |   0.02   
--------------------
 1509   |   0.02   
--------------------
 1510   |   0.02   
--------------------
 1511   |   0.02   
--------------------
 1512   |   0.02   
--------------------
 1513   |   0.02   
--------------------
 1514   |   0.02   
--------------------
 1515   |   0.02   
--------------------
 1516   |   0.02   
--------------------
 1517   |   0.03   
--------------------
 1518   |   0.02   
--------------------
 1519   |   0.02   
--------------------
 1520   |   0.02   
--------------------
 1521   |   0.02   
--------------------
 1522   |   0.02   
--------------------
 1523   |   0.02   
--------------------
 1524   |   0.02   
--------------------
 1525   |   0.02

rate limit exceeded: sleeping 102.48783278465271 secs


 1801   |   0.02   
--------------------
 1802   |   0.02   
--------------------
 1803   |   0.02   
--------------------
 1804   |   0.02   
--------------------
 1805   |   0.02   
--------------------
 1806   |   0.02   
--------------------
 1807   |   0.02   
--------------------
 1808   |   0.05   
--------------------
 1809   |   0.02   
--------------------
 1810   |   0.02   
--------------------
 1811   |   0.02   
--------------------
 1812   |   0.02   
--------------------
 1813   |   0.02   
--------------------
 1814   |   0.02   
--------------------
 1815   |   0.02   
--------------------
 1816   |   0.02   
--------------------
 1817   |   0.02   
--------------------
 1818   |   0.02   
--------------------
 1819   |   0.02   
--------------------
 1820   |   0.02   
--------------------
 1821   |   0.02   
--------------------
 1822   |   0.03   
--------------------
 1823   |   0.02   
--------------------
 1824   |   0.02   
--------------------
 1825   |   0.02

rate limit exceeded: sleeping 137.10561752319336 secs


 2101   |   0.02   
--------------------
 2102   |   0.02   
--------------------
 2103   |   0.02   
--------------------
 2104   |   0.02   
--------------------
 2105   |   0.02   
--------------------
 2106   |   0.02   
--------------------
 2107   |   0.02   
--------------------
 2108   |   0.02   
--------------------
 2109   |   0.02   
--------------------
 2110   |   0.02   
--------------------
 2111   |   0.02   
--------------------
 2112   |   0.02   
--------------------
 2113   |   0.02   
--------------------
 2114   |   0.02   
--------------------
 2115   |   0.02   
--------------------
 2116   |   0.02   
--------------------
 2117   |   0.02   
--------------------
 2118   |   0.02   
--------------------
 2119   |   0.02   
--------------------
 2120   |   0.02   
--------------------
 2121   |   0.02   
--------------------
 2122   |   0.02   
--------------------
 2123   |   0.02   
--------------------
 2124   |   0.02   
--------------------
 2125   |   0.02